In [1]:
# !pip install numpy
# !pip install matplotlib
# !pip install scipy

In [2]:
import numpy as np
import math
from scipy.optimize import linear_sum_assignment

In [3]:
requests = [
    10.0, 10.909090909090908, 11.818181818181818, 12.727272727272727,
    13.636363636363637, 14.545454545454545, 15.454545454545453,
    16.363636363636363, 17.272727272727273, 18.18181818181818,
    19.09090909090909, 20.0, 20.909090909090907, 21.81818181818182,
    22.727272727272727, 23.636363636363637, 24.545454545454547,
    25.454545454545453, 26.363636363636363, 27.272727272727273,
    28.18181818181818, 29.09090909090909, 30.0, 30.90909090909091,
    31.818181818181817, 32.72727272727273, 33.63636363636364, 34.54545454545455,
    35.45454545454545, 36.36363636363636, 37.27272727272727, 38.18181818181818,
    39.09090909090909, 40.0, 40.90909090909091, 41.81818181818181,
    42.72727272727273, 43.63636363636363, 44.54545454545455, 45.45454545454545,
    46.36363636363636, 47.27272727272727, 48.18181818181818, 49.090909090909086,
    50.0, 50.90909090909091, 51.81818181818182, 52.72727272727273,
    53.63636363636363, 54.54545454545455, 55.45454545454545, 56.36363636363636,
    57.27272727272727, 58.18181818181818, 59.090909090909086, 60.0,
    60.90909090909091, 61.81818181818181, 62.72727272727273, 63.63636363636363,
    64.54545454545455, 65.45454545454545, 66.36363636363636, 67.27272727272728,
    68.18181818181819, 69.0909090909091, 70.0, 70.9090909090909,
    71.81818181818181, 72.72727272727272, 73.63636363636363, 74.54545454545455,
    75.45454545454545, 76.36363636363636, 77.27272727272727, 78.18181818181817,
    79.0909090909091, 80.0, 80.9090909090909, 81.81818181818181,
    82.72727272727272, 83.63636363636364, 84.54545454545455, 85.45454545454545,
    86.36363636363636, 87.27272727272727, 88.18181818181817, 89.0909090909091,
    90.0, 90.9090909090909, 91.81818181818181, 92.72727272727272,
    93.63636363636364, 94.54545454545455, 95.45454545454545, 96.36363636363636,
    97.27272727272727, 98.18181818181817, 99.0909090909091, 100.0
  ]

height = round(math.sqrt(1.05 * np.sum(requests) / 1.6), 2)
width = round(height * 1.6, 2)

In [4]:
def factor_pairs(x):
    pairs = []
    limit = int(abs(x) ** 0.5) + 1
    for i in range(1, limit):
        if x % i == 0:
            pairs.append((i, x // i))
    return pairs


def calculate_piece_areas(x_cuts, y_cuts):
    x_coords = np.sort(np.concatenate(([0], x_cuts, [width])))
    y_coords = np.sort(np.concatenate(([0], y_cuts, [height])))
    
    piece_widths = np.diff(x_coords)
    piece_heights = np.diff(y_coords)
    
    areas = np.concatenate(np.outer(piece_widths, piece_heights))
    
    return areas


def loss_function(areas, requests):
    R = requests
    V = areas

    num_requests = len(R)
    num_values = len(V)

    cost_matrix = np.zeros((num_requests, num_values))

    for i, r in enumerate(R):
        for j, v in enumerate(V):
            cost_matrix[i][j] = abs(r - v) / r

    row_indices, col_indices = linear_sum_assignment(cost_matrix)

    total_cost = sum(
        cost_matrix[row_indices[i], col_indices[i]] for i in range(len(row_indices))
    )

    return total_cost

In [5]:
factors = factor_pairs(len(requests))

num_horizontal, num_vertical = factors[3]
num_horizontal, num_vertical

(5, 20)

In [6]:
x_cuts = np.random.randint(1, width, num_vertical)
y_cuts = np.random.randint(1, height, num_horizontal)

areas = calculate_piece_areas(x_cuts, y_cuts)
loss = loss_function(areas, requests)

print("Areas of individual pieces:", areas)
print("Total loss:", loss)

Areas of individual pieces: [ 21.     259.      42.      35.      49.      14.56     6.      74.
  12.      10.      14.       4.16    18.     222.      36.      30.
  42.      12.48     6.      74.      12.      10.      14.       4.16
   3.      37.       6.       5.       7.       2.08    36.     444.
  72.      60.      84.      24.96    24.     296.      48.      40.
  56.      16.64     3.      37.       6.       5.       7.       2.08
   3.      37.       6.       5.       7.       2.08    42.     518.
  84.      70.      98.      29.12     0.       0.       0.       0.
   0.       0.      18.     222.      36.      30.      42.      12.48
   3.      37.       6.       5.       7.       2.08    12.     148.
  24.      20.      28.       8.32     0.       0.       0.       0.
   0.       0.       9.     111.      18.      15.      21.       6.24
   3.      37.       6.       5.       7.       2.08     9.     111.
  18.      15.      21.       6.24    54.     666.     108.      90

In [7]:
def calculate_gradient(x_cuts, y_cuts, requests, curr_loss, epsilon=1):
    grad_x_cuts = np.zeros_like(x_cuts, dtype=float)
    grad_y_cuts = np.zeros_like(y_cuts, dtype=float)
    
    for i in range(len(x_cuts)):
        x_cuts_eps = x_cuts.copy()
        x_cuts_eps[i] += epsilon
        areas_eps = calculate_piece_areas(x_cuts_eps, y_cuts)
        loss_eps = loss_function(areas_eps, requests)
        grad_x_cuts[i] = (loss_eps - curr_loss) / epsilon
    
    for i in range(len(y_cuts)):
        y_cuts_eps = y_cuts.copy()
        y_cuts_eps[i] += epsilon
        areas_eps = calculate_piece_areas(x_cuts, y_cuts_eps)
        loss_eps = loss_function(areas_eps, requests)
        grad_y_cuts[i] = (loss_eps - curr_loss) / epsilon
    
    return grad_x_cuts, grad_y_cuts

x_cuts = np.random.randint(1, width, num_vertical)
y_cuts = np.random.randint(1, height, num_horizontal)
curr_loss = loss_function(calculate_piece_areas(x_cuts, y_cuts), requests)
grad_x_cuts, grad_y_cuts = calculate_gradient(x_cuts, y_cuts, requests, curr_loss)

print("Gradient for x_cuts:", grad_x_cuts)
print("Gradient for y_cuts:", grad_y_cuts)

Gradient for x_cuts: [ 0.58887516 -0.77474047 -0.86327422  0.65268526  1.02524924  0.13520631
 -0.69788283 -0.04136059 -1.08467283  0.58887516  0.40131462  0.
  0.40131462  0.58887516 -0.011618   -0.69788283 -1.13233723  0.94471443
  0.99371845  1.56634429]
Gradient for y_cuts: [-1.8960522   0.5977491   1.2836681  -0.25616279 -0.37181137]
